Transpose Conv Operation Simple Explanation

In [2]:
import numpy as np

In [3]:
input = np.array([[4, 5, 8, 7], [1, 8, 8, 8], [3, 6, 6, 4], [6, 5, 7, 8]], np.float32)
print(input)

[[4. 5. 8. 7.]
 [1. 8. 8. 8.]
 [3. 6. 6. 4.]
 [6. 5. 7. 8.]]


In [4]:
kernel = np.array([[1, 4, 1], [1, 4, 3], [3, 3, 1]], np.float32)
print(kernel)

[[1. 4. 1.]
 [1. 4. 3.]
 [3. 3. 1.]]


In [5]:
output = np.zeros((2, 2), np.float32)
print(output)

[[0. 0.]
 [0. 0.]]


In [6]:
# conv operation
for i in range(2):
    for j in range(2):
        output[i][j] = np.sum(input[i:i+3, j:j+3] * kernel)
print(output)

[[122. 148.]
 [126. 134.]]


In [7]:
# construct a conv matrix that produces the same result as the kernel
# Padding the kernel to a matrix with the shape of (4 = 2x2, 16 = 4x4)
kernel_matrix = np.array([[1, 4, 1, 0, 1, 4, 3, 0, 3, 3, 1, 0, 0, 0, 0, 0], [0, 1, 4, 1, 0, 1, 4, 3, 0, 3, 3, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 4, 1, 0, 1, 4, 3, 0, 3, 3, 1, 0], [0, 0, 0, 0, 0, 1, 4, 1, 0, 1, 4, 3, 0, 3, 3, 1]], np.float32)
print(kernel_matrix)

[[1. 4. 1. 0. 1. 4. 3. 0. 3. 3. 1. 0. 0. 0. 0. 0.]
 [0. 1. 4. 1. 0. 1. 4. 3. 0. 3. 3. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 4. 1. 0. 1. 4. 3. 0. 3. 3. 1. 0.]
 [0. 0. 0. 0. 0. 1. 4. 1. 0. 1. 4. 3. 0. 3. 3. 1.]]


In [8]:
# obtain the same result
output2 = np.matmul(kernel_matrix, input.reshape(16, 1)).reshape(2, 2)
print(output2)
print(output == output2)

[[122. 148.]
 [126. 134.]]
[[ True  True]
 [ True  True]]


The Conv Operation actually produces a 2x2=4x1 matrix from 4x4=16x1 matrix which can be implemented with (4, 16) matrix by matrix multiplication.
Thus, a (16, 4) matrix can also produces a 4x4=16x1 matrix from 2x2=4x1 matrix and this is the transpose convolution.

## Look at what torch does

In [2]:
import torch
import numpy as np

cnn = torch.nn.Conv2d(1, 1, 3, 1, 0, bias=False)
kernel = np.array([[1, 4, 1], [1, 4, 3], [3, 3, 1]], np.float32)
def init_weights(m):
    class_name = m.__class__.__name__
    if class_name.find('Conv') != -1:
        m.weight.data.copy_(torch.from_numpy(kernel))

cnn.apply(init_weights)
print('cnn.weight:')
print(cnn.weight)
input = np.array([[4, 5, 8, 7], [1, 8, 8, 8], [3, 6, 6, 4], [6, 5, 7, 8]], np.float32)
img = torch.from_numpy(input).unsqueeze(0).unsqueeze(0)
cnn(img)

cnn.weight:
Parameter containing:
tensor([[[[1., 4., 1.],
          [1., 4., 3.],
          [3., 3., 1.]]]], requires_grad=True)


tensor([[[[122., 148.],
          [126., 134.]]]], grad_fn=<ThnnConv2DBackward>)

The result is the same.

### Transposed Convolution in PyTorch

In [31]:
# pytorch
tcnn = torch.nn.ConvTranspose2d(1, 1, 3, 1, 0, bias=False)
kernel = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], np.float32)
def init_weights(m):
    class_name = m.__class__.__name__
    if class_name.find('Conv') != -1:
        m.weight.data.copy_(torch.from_numpy(kernel))

tcnn.apply(init_weights)
print('tcnn.weight:')
print(tcnn.weight)
input = np.array([[1, 2], [3, 4]], np.float32)
print('input:')
print(input)
img = torch.from_numpy(input).unsqueeze(0).unsqueeze(0)
tcnn(img)

tcnn.weight:
Parameter containing:
tensor([[[[1., 2., 3.],
          [4., 5., 6.],
          [7., 8., 9.]]]], requires_grad=True)
input:
[[1. 2.]
 [3. 4.]]


tensor([[[[ 1.,  4.,  7.,  6.],
          [ 7., 23., 33., 24.],
          [19., 53., 63., 42.],
          [21., 52., 59., 36.]]]], grad_fn=<ThnnConvTranspose2DBackward>)

In [45]:
# Mine
print('input:')
print(input)

print('kernel:')
print(kernel)
output = np.zeros((4, 4), np.float32)
# traverse each element of input image
for i in range(2):
    for j in range(2):
        output[i:i+3, j:j+3] += input[i, j] * kernel
print('output:')
print(output)

input:
[[1. 2.]
 [3. 4.]]
kernel:
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
output:
[[ 1.  4.  7.  6.]
 [ 7. 23. 33. 24.]
 [19. 53. 63. 42.]
 [21. 52. 59. 36.]]


So PyTorch simple multiplies each element in the input with the kernel and add them together.